In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Transformer
- Transformer은 input sentence를 넣어서 output sentence를 출력해 내는 구조이다
- 크게 Encoder, Decoder 구조로 이루어져 있다
- Sequential Data를 다루며 병렬 처리를 할 수 있다는 점이 가장 큰 contribution

In [2]:
class Transformer(nn.Module):
    
    def __init__(self, encoder, decoder):
        super(Transformer, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self,x,s):
        encoder_out = self.encoder(x)
        decoder_out = self.decoder(s, encoder_out)
        
        return decoder_out
           

## Encoder
- Encoder는 Encoder layer가 n개 쌓여있는 구조(논문에서는 n=6)
- Encoder layer의 input과 output의 shape은 동일하다
- Encoder layer는 input으로 들어오는 vector에 대해서 더 높은 차원에서의 context를 담는다.

In [3]:
class Encoder(nn.Module):
    
    def __init__(self, encoder_layer, n_layers):
        super(Encoder, self).__init__()
        self.encoder = []
        for i in range(n_layers):
            self.encoder.append(copy.deepcopy(self.encoder_layer))
            
    def forward(self, x):
        out = x
        for layer in self.encoder:
            out = layer(out)
            
        return out

## Encoder layer
- Encoder layer는 크기 multi-head-attention-layer와 Position-wise-feed-forward-layer로 이루어져 있다.
- 중간중간엔 add&norm layer가 끼어져 있다


In [4]:
class EncoderLayer(nn.Module):
    def __init__(self, multi_head_attention_layer, position_wise_feed_forward_layer):
        super(EncoderLayer, self).__init__()
        
        self.multi_head_attention_layer = multi_head_attention_layer
        self.position_wise_feed_forward_layer = position_wise_feed_forward_layer
        
    def forward(self, x):
        out = self.multi_head_attention_layer(x)
        out = self.position_wise_feed_forward_layer(out)
        
        return out

## Self Attention
- Attention이라는 것은 넓은 범위의 전체 data에 대해서 특정한 부분에 집중한다는 의미이다
- Self-Attention은 문장에 token이 n개 있다고 가정할 경우, n×n 번 연산을 수행해 모든 token들 사이의 관계를 직접 구해낸다. 중간의 다른 token들을 거치지 않고 바로 direct한 관계를 구하는 것이기 때문에 Recurrent Network에 비해 더 명확하게 관계를 잡아낼 수 있다.
- Query, Key, Value의 3가지 vector를 사용한다.
- Query는 현재 시점의 token
- Key, Value는 구하고자 하는 대상의 token(의미적으로는 같지만 다른 fc layer를 거치기 때문에 다른 값을 가지고 있다)
- Query Attention(Q,K,V) = softmax(QK^T/sqrt(D_k))V
- (batch, seq_len, d_emb)의 shape을 가진 text 토큰들이있다. 이들은 fc layer를 거쳐 (batch, seq_len, d_k)의 shape을 가진 Q,K,V vector로 변환한다, 이후 Q.matmul(k.T)를 거쳐 (batch,seq_len,seq_len)구조의 attention score를 만들어내고, mask_padding과 softmax, norm과정을 거친 후 V와 matmul을 진행해준다. 최종적으로는 input shape과 같은 (batch, seq_len, d_emb)형태의 vector가 출력된다.
 

In [5]:
def calculate_attention(self, query, key, value, mask):
    d_k = key.size(-1)
    attention_score = query.matmul(key.transpose(-2,-1))
    attention_score = attention_score / math.sqrt(d_k) ## scaling
    
    if mask is not None: ## masking(masking은 대개 scaling과 softmax사이에 이루어진다)
        attention_score = score.masked_fill(mask==0, -1e9)
        
    attention_prob = F.softmax(score, dim = -1)
    
    output = attention_prob.matmul(value)
    
    return output

## Multi Head Attention
- encoder layer에서 h개의 self attention을 진행해 그 결과를 종합하여 사용하는 것
- d_model = h*d_k  
- (seq_len, d_k)의 q,k,v를 h번 연산하는것이 아닌 fc layer를 통해 (seq_len, d_model)의 q,k,v를 생성하여 한번에 계산한다
- Multi-Head Attention Layer의 개념적인 의미는 사실 단지 dk의 크기를 dmodel로 확장시키는 단순한 구현으로 끝난다는 점이다

In [6]:
class MultiheadAttention(nn.Module):
    def __init__(self, d_model, h, qkv_fc_layer, fc_layer):
        super(MultiheadAttention, self).__init__()
        
        self.d_model = d_model
        self.h = h
        self.query_fc_layer = copy.deepcopy(qkv_fc_layer)
        self.key_fc_layer = copy.deepcopy(qkv_fc_layer)
        self.value_vc_layer = copy.deepcopy(qkv_vc_layer)
        self.fc_layer = fc_layer
        
        
    def forward(self, query, key, value, mask=None): ## Q,K,V는 실제 q,k.v vector가 아닌 sentence matrix이다
        
        n_batch = query.shape[0]
        
        def transform(x, fc_layer): ## (batch, len_seq, n_emb) ==> (batch, h, len_seq, d_k)
            out = fc_layer(x) ## (batch, len_seq, n_model)
            out = out.view(n_batch, -1, self.h, self.d_model/self.h) ## (batch, len_seq, h, d_k)
            out = out.transpose(1,2) ## (batch, h, len_seq, d_k)  ==> self attention진행시 d_k를 기준으로 q.k의 행렬곱이 이루어지므로 마지막 차원은 d_k이여야 한다
                                                                # ==> 또한 mask matrix의 shape이 (batch, seq_len, seq_len)이므로 -2번째 shape은 seq_len과 같아야 한다           
            return out
        
        query = transform(query, self.query_fc_layer)
        key = transform(key, self.key_fc_layer)
        value = transform(value, self.value_fc_layer)
        
        
        if mask is not None:
            mask = mask.unsqueeze(1) ## (batch, 1 , len_seq, len_seq)
            
        out = self.cacluate_attention(query, key, value, mask) ## (batch, h, len_seq, d_k)
        out = out.transpose(1,2) ## (batch, len_seq, h, d_k)
        out = contiguous().view(n_batch, -1, self.d_model) ##(batch, len_seq, d_model)
        out = self.fc_layer(out) ## (batch, len_seq, d_emb)
        
        return out
         

## Encoder layer(2)

In [7]:
class EncoderLayer(nn.Module):
    def __init__(self, multi_head_attention_layer, position_wise_feed_forward_layer):
        super(EncoderLayer, self).__init__()
        
        self.multi_head_attention_layer = multi_head_attention_layer
        self.position_wise_feed_forward_layer = position_wise_feed_forward_layer
        
    def forward(self, x, mask):
        out = self.multi_head_attention_layer(query=x, key=x, value=x, mask=mask)
        out = self.position_wise_feed_forward_layer(out)
        
        return out

## Encoder(2)

In [8]:
class Encoder(nn.Module):
    
    def __init__(self, encoder_layer, n_layers):
        super(Encoder, self).__init__()
        self.encoder = []
        for i in range(n_layers):
            self.encoder.append(copy.deepcopy(self.encoder_layer))
            
    def forward(self, x, maks):
        out = x
        for layer in self.encoder:
            out = layer(out,mask)
            
        return out

## Tranformer(2)

In [9]:
class Transformer(nn.Module):
    
    def __init__(self, encoder, decoder):
        super(Transformer, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self,src, trg, mask):
        encoder_out = self.encoder(src,mask)
        decoder_out = self.decoder(trg, encoder_out)
        
        return decoder_out
           

## Position wise feed forward layer
- 단순히 2개의 fc layer를 가진다 (d_emb, d_ff) (d_ff, d_emb)


In [11]:
class PositionWiseFeedForwardLayer(nn.Module):
    
    def __init__(self, first_layer, second_layer):
        super(PositionWiseFeedForwardLayer, self).__init__()
        
        self.first_layer = first_layer
        self.second_layer = second_layer
        
    def forward(self,x):
        out = self.first_layer(x)
        out = F.relu(out)
        out = dropout(out)
        out = self.second_layer(out)
        
        return out
        

## Norm layer
- MultiHeadAttentionLayer와 PositionWiseFeedForwardLayer은 residual connection로 둘러쌓여져 있다
- residual connection이란 y = f(x)+x를 의미한다
- 즉 output만을 사용하는 것이 아닌 output+input을 사용한다 이를 통해서 back propagation 진행시 발생할 수 있는 gradient vanishing문제를 해결할 수 있다

In [12]:
class ResidualConnectionLayer(nn.Module):
    def __init__(self, norm_layer):
        super(REsidualConnectionLayer, self).__init__()
        
        self.norm_layer = norm_layer
        
        
    def forward(self,x, sub_layer):
        out = sub_layer(x) + x
        out = self.norm_layer(out)
        
        return out

## Final Encoder layer 

In [14]:
class EncoderLayer(nn.Module):
    def __init__(self, multi_head_attention_layer, position_wise_feed_forward_layer, norm_layer):
        super(EncoderLayer, self).__init__()
        
        self.multi_head_attention_layer = multi_head_attention_layer
        self.position_wise_feed_forward_layer = position_wise_feed_forward_layer
        self.rcl = [self.ResidualConnectionLayer(copy.deepcopy(norm_layer)) for i in range(2)]
        
        
    def forward(self, x, mask):
        out = self.rcl[0](x, lambda x : self.multi_head_attention_layer(query=x, key=x, value=x, mask=mask))
        out = self.rcl[1](x, lambda x : self.position_wise_feed_forward_layer(x))
        
        return out

## Decoder
- context vector와 right shifted sentence를 input으로 받아서 sentence를 output으로 생성한다(Teacher forcing)
- right shifted sentence ==> ground truth[:-1]을 뜻한다
- Teacher forcing in transformer ==> subsequent masking
- 병렬 연산을 위해 ground truth의 embedding을 matrix로 만들어 input으로 그대로 사용하게 되면, Decoder에서 Self-Attention 연산을 수행하게 될 때 현재 출력해내야 하는 token의 정답까지 알고 있는 상황이 발생한다. 따라서 masking을 적용해야 한다. i번째 token을 생성해낼 때, 1∼i−1의 token은 보이지 않도록 처리를 해야 하는 것이다. 이러한 masking 기법을 subsequent masking이라고 한다.
- 마찬가지로 n개의 decoder layer로 구성되어 있고 각각의 decoder layer는 MaskedMultiHeadAttentionLayer - MultiHeadAttentionLayer - FeedForwardLayer로 이루어져 있다   

In [29]:
def subsequent_mask(size):
    atten_shape = (1, size, size)
    mask = np.triu(np.ones(atten_shape).astype('uint8'), k=1)
    return torch.from_numpy(mask) == 0

def make_std_mask(tgt, pad):
    tgt_mask = (tgt!=pad)
    tgt_mask = tgt_mask.unsqueeze(-2)
    tgt_mask = tgt_mask&Variable(subsequent_mask(tgt.size(-1)).type_as(tgt.data))
    
    return tgt_mask

## Transformer(3)

In [30]:
class Transformer(nn.Module):
    
    def __init__(self, encoder, decoder):
        super(Transformer, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self,src, trg, src_mask, trg_mask):
        encoder_out = self.encoder(src,src_mask)
        decoder_out = self.decoder(trg, trg_mask, encoder_out)
        
        return decoder_out
           
        

## Masked multi head attention layer
- mask로 들어오는 인자가 pad masking + subsequent masking이라는 점

## Mult head attention layer
- masked multi head attention layer의 output + encoder의 context vector를 input으로 받는다
- teaching forcing을 통해 건너온 masked multihead attenition layer의 output을 Query로 encoder의 context vector를 key, value로 삼는다 
- 즉 Decodr layer에서의 multi head attention layer는 label data와 input data와의 attention을 구하는 구간이다

## Decoder

In [48]:
class Decoder(nn.Module):
    
    def __init__(self, decoder_layer, n_layers):
        super(Decoder, self).__init__()
        self.decoder=[]
        for i in range(n_layers):
            self.decoder.append(copy.deepcopy(decoder_layer))
            
    def forward(self, mask, encoder_output, encoder_mask):
        out = x 
        for layer in decoder:
            out = layer(out, mask, encoder_output, encoder_mask)
            
        return out

## Decoder layer

In [33]:
class DecoderLayer(nn.Module):
    
    def __init__(self, masked_multi_head_attention_layer, multi_head_attention_layer, position_wise_feed_forward_layer, norm_layer):
        super(DecoderLayer, self).__init__()
        
        self.masked_multi_head_attention_layer = ResidualConnectionLayer(masked_multi_head_attention_layer, copy.deepcopy(norm_layer))
        self.multi_head_attention_layer = ResidualConnectionLayer(multi_head_attention_layer, copy.deepcopy(norm_layer))
        self.position_wise_feed_forward_layer = ResidualConnectionLayer(position_wise_feed_forward_layer, copy.deepcopy(norm_layer))
        
        
    def forward(self, x, mask, encoder_output, encoder_mask):
        out = self.masked_multi_head_attention_layer(query=x, key=x, value=x, mask=mask)
        out = self.multi_head_attention_layer(query=out, key= encoder_output, value=encoder_output, mask= encoder_mask)
        out = self.position_wise_feed_forward_layer(x=out)
        
        return out
    

## Transformer(4)

In [34]:
class Transformer(nn.Module):
    
    def __init__(self, encoder, decoder):
        super(Transformer, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self,src, trg, src_mask, trg_mask):
        encoder_out = self.encoder(src,src_mask)
        decoder_out = self.decoder(trg, trg_mask, encoder_out, src_mask)
        
        return decoder_out
           
        

## Transformer input
- Embedding + Positional_Encoding

In [44]:
class TransformerEmbedding(nn.Module):
    def __init__(self, embedding, positional_encoding):
        super(TransformerEncoding, self).__init__()
        
        self.embedding = nn.Sequential(embedding, positional_encoding) 
        
    def forward(self ,x):
        embedding = self.embedding(x)
        
        return embedding

## Embedding
- 단순 embedding + scaling(sqrt(n_emb)를 곱해주기)

In [38]:
class Embedding(nn.Module):
    def __init__(self, embedding, vocab, n_emb):
        super(Embedding, self).__init__()
        
        self.embedding = nn.Embedding(len(vocab), n_emb)
        self.vocab = vocab
        self.n_emb = n_emb
        
    def forward(self, x):
        out = self.embedding(x) * math.sqrt(slef.n_emb) ## scaling
        return out
        
        

##  Positional Encoding 
-  PositionalEncoding의 목적은 positional정보(대표적으로 token의 순서, 즉 index number)를 정규화시키기 위한 것이다
- sin 함수와 cos함수를 사용하는데, 짝수 index에는 sin함수를, 홀수 index에는 cos함수를 사용하게 된다. 이를 사용할 경우 항상 -1에서 1 사이의 값만이 positional 정보로 사용되게 된다.

In [43]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_embed, max_seq_len=5000):
        super(PositionalEncoding, self).__init__()
        encoding = torch.zeros(max_seq_len, d_embed)
        position = torch.arange(0, max_seq_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_embed, 2) * -(math.log(10000.0) / d_embed))
        encoding[:, 0::2] = torch.sin(position * div_term)
        encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = encoding

    def forward(self, x):
        out = x + Variable(self.encoding[:, :x.size(1)], requires_grad=False) ## positional encoding은 학습되는 parameter가 아니다!
        out = self.dropout(out)
        return out

## Transformer(5)

In [40]:
class Transformer(nn.Module):
    
    def __init__(self, src_emb, trg_emb, encoder, decoder_out):
        super(Transformer, self).__init__()
        
        self.src_emb = src_emb
        self.trg_emb = trg_emb
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self,src, trg, src_mask, trg_mask):
        encoder_out = self.encoder(self.src_emb(src),src_mask)
        decoder_out = self.decoder(self.trc_emb(trg), trg_mask, encoder_out, src_mask)
        
        return decoder_out
           
        

## After Decoder (generater)
- Decoder의 output이 그대로 Transformer의 최종 output이 되는 것은 아니다. 추가적인 layer를 거쳐간다. 이 layer들을 generator라고 부른다.
- 우리가 결국 해내고자 하는 목표는 Decoder의 output이 sentence, 즉 token의 sequence가 되는 것이다. 그런데 Decoder의 output은 그저 (n_batch×seq_len×d_model)의 shape를 갖는 matrix일 뿐이다. 이를 vocabulary를 사용해 실제 token으로 변환할 수 있도록 차원을 수정해야 한다. 따라서 FC Layer를 거쳐 마지막 dimension을 d_model에서 len(vocab)으로 변경한다. 그래야 실제 vocabulary 내 token에 대응시킬 수 있는 값이 되기 때문이다. 이후 softmax 함수를 사용해 각 vocabulary에 대한 확률값으로 변환하게 되는데, 이 때 log_softmax를 사용해 성능을 향상시킨다.

In [51]:
class Transformer(nn.Module):
    
    def __init__(self, src_emb, trg_emb, encoder, decoder, fc_layer):
        super(Transformer, self).__init__()
        
        self.src_emb = src_emb
        self.trg_emb = trg_emb
        self.encoder = encoder
        self.decoder = decoder
        self.fc_layer = fc_layer 
        
    def forward(self,src, trg, src_mask, trg_mask):
        encoder_out = self.encoder(self.src_emb(src),src_mask)
        decoder_out = self.decoder(self.trc_emb(trg), trg_mask, encoder_out, src_mask)
        out = self.fc_layer(decoder_out)
        out = F.log_softmax(out, dim=-1)    
        
        return out
           
        

## Make Model

In [57]:
def make_model(src_vocab, trg_vocab, n_emb=512, n_layer=6, d_model=512, h=8, d_ff=2048):
    
    cp = lambda x : copy.deepcopy(x)
    
    multi_head_attention_layer = MultiHeadAttentionLayer(d_model= d_model, 
                                                         h=h, 
                                                         qkv_fc_layer = nn.Linear(n_emb, d_model),
                                                         fc_layer = nn.Linear(d_model, n_emb)) 
    
    position_wise_feed_forward_layer = PositionWiseFeedForwardLayer(first_layer=nn.Linear(n_emb, d_ff),
                                                                   second_layer=nn.Linear(d_ff, n_emb))
                                            
        
    norm_layer = nn.LayerNorm(n_emb, eps=1e-6)
    

    model = Transformer(
        src_emb = TransformerEmbedding(embedding = Embedding(n_emb=n_emb,vocab=src_vocab),
                                       positional_encoding = PositionalEncoding(n_emb=n_emb)),
        
        trg_emb = TransformerEmbedding(embedding= Embedding(n_emb=n_emb, vocab= trg_vocab),
                                       positional_encoding=  PositionalEncoding(n_emb=n_emb)),
        
        encoder = Encoder(encoder_layer = EncoderLayer(multi_head_attention_layer=cp(multi_head_attention_layer), 
                                                       position_wise_feed_forward_layer=cp(position_wise_feed_forward_layer), 
                                                       norm_layer = cp(norm_layer)),n_layers=6),
        
        decoder = Decoder(decoder_layer= DecoderLayer(masked_multi_head_attention_layer = cp(masked_multi_head_attention_layer),
                                                   multi_head_attention_layer=cp(multi_head_attention_layer),
                                                   position_wise_feed_forward_layer=cp(position_wise_feed_forward_layer), 
                                                   norm_layer=cp(norm_layer)), n_layers = 6),
        
        fc_layer = nn.Linear(d_model, len(trg_vocab)))
                                      
    
    return model